<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización


In [504]:
import numpy as np
import pandas as pd
from collections import Counter
from numpy.linalg import norm

In [505]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [506]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

#### Defino mis funciones utiles

In [507]:
def getDocuments(corpus: list, getAsListofLists = True):
    '''Recibe una lista de documentos y retorna una lista de listas de documentos'''
    if getAsListofLists:
        return [[document] for document in corpus]
    return [document for document in corpus]

def getTerms(document: str , eliminateDuplicates = False):
    """recibe un documento tipo list y retorna una lista de palabras"""
    l = document.split(' ')
    if eliminateDuplicates:
        l = list(set(l))
    return l

def getCorpusWords(corpus: list):
    result = ''
    for v in corpus:
        result+= v+' '
    return getTerms(result.strip(),True)


In [508]:
# Las testeo:
documents = getDocuments(list(corpus))
terms = getTerms(documents[0][0])
corpusWords = getCorpusWords(list(corpus))

print('documents',documents)
print('terms',terms)
print('corpusWords',corpusWords)

documents [['que dia es hoy'], ['martes el dia de hoy es martes'], ['martes muchas gracias']]
terms ['que', 'dia', 'es', 'hoy']
corpusWords ['muchas', 'gracias', 'de', 'martes', 'hoy', 'es', 'el', 'dia', 'que']


### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [509]:
# Cada documento transformarlo en una lista de términos

doc2termsList = []
for document in getDocuments(list(corpus)):
    doc2termsList.append(getTerms(document[0]))
print(doc2termsList)


[['que', 'dia', 'es', 'hoy'], ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'], ['martes', 'muchas', 'gracias']]


In [510]:
# Armar un vector de términos no repetidos de todos los documentos
corpusWords = getCorpusWords(list(corpus))
print(corpusWords)


['muchas', 'gracias', 'de', 'martes', 'hoy', 'es', 'el', 'dia', 'que']


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [511]:
# Suponiendo lista de textos senteces y palabras words
documents = getDocuments(list(corpus),False)
ds = pd.DataFrame({"Textos": documents})
ds['Textos'] = ds['Textos']
ds.head()

,Textos
0,que dia es hoy
1,martes el dia de hoy es martes
2,martes muchas gracias


In [512]:
def oneHot(terms,documents):
    matriz = np.zeros((len(documents), len(terms)))
    for i, t in enumerate(documents):
        for j, p in enumerate(terms):
            if p in t:
                matriz[i, j] = 1
    matriz = matriz.astype(int)
    return pd.DataFrame(matriz, columns=terms)


terms = getCorpusWords(list(corpus))
documents = getDocuments(list(corpus),False)

df_concat = pd.concat([ds, oneHot(terms,documents)], axis=1, join='outer')

#tabla final
df_concat.head()

,Textos,muchas,gracias,de,martes,hoy,es,el,dia,que
0,que dia es hoy,0,0,0,0,1,1,0,1,1
1,martes el dia de hoy es martes,0,0,1,1,1,1,1,1,0
2,martes muchas gracias,1,1,0,1,0,1,0,0,0


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [513]:
def getFrec(texts: list, getwords):
    words = getwords(texts)
    df = pd.DataFrame(columns=words)
    wordsFre = []
    for i,text in enumerate(texts):
        for word in text.split(' '):
            df.loc[i] = [text.split(' ').count(word) for word in words]
    df_concat = pd.concat([pd.DataFrame(texts,columns=['Texts']),df ], axis=1, join='outer')
    return df_concat

texts = getDocuments(list(corpus),False)
getFrec(texts,getCorpusWords).head()


,Texts,muchas,gracias,de,martes,hoy,es,el,dia,que
0,que dia es hoy,0,0,0,0,1,1,0,1,1
1,martes el dia de hoy es martes,0,0,1,2,1,1,1,1,0
2,martes muchas gracias,1,1,0,1,0,0,0,0,0


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [514]:
   
def calcular_idf(documentos):
    # Crear un conjunto de términos únicos en todos los documentos
    terminos_unicos = set()
    for documento in documentos:
        terminos_unicos.update(documento.split())
    # Calcular el IDF para cada término
    num_documentos = len(documentos)
    idf = {}
    for termino in terminos_unicos:
        num_documentos_con_termino = sum(termino in documento for documento in documentos)
        # Calculo el IDF
        idf[termino] = np.log(num_documentos / (num_documentos_con_termino))
    
    return idf

def calcular_tf(documento):
    terminos = documento.split()
    contador = Counter(terminos)
    # Calcular la frecuencia de términos normalizada
    total_terminos = len(terminos)
    tf = {termino: frecuencia/total_terminos for termino, frecuencia in contador.items()}
    
    return tf
            
def calcular_tfidf(documentos):

    # Calculo todos los tf
    tfs = [calcular_tf(documento) for documento in documentos]
    # Calculo todos los idf
    idf = calcular_idf(documentos)
    # Calculo la representación TF-IDF para cada documento
    tfidf = {}
    for doc in documentos:
        for tf in tfs:
            tfidf_documento = {termino: tf[termino] * idf[termino] for termino in tf}
            tfidf[doc] = list(tfidf_documento.values())

    # Retorno una matriz/lista para cada uno de los textos
    return tfidf


In [515]:
texts = getDocuments(list(corpus),False)
tfidfs = calcular_tfidf(texts)
print(tfidfs)

{'que dia es hoy': [0.13515503603605478, 0.3662040962227032, 0.3662040962227032], 'martes el dia de hoy es martes': [0.13515503603605478, 0.3662040962227032, 0.3662040962227032], 'martes muchas gracias': [0.13515503603605478, 0.3662040962227032, 0.3662040962227032]}


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [516]:

def calcSimCos(tfidf_ref,tfidf_comp):
    tfidf_ref = np.array([1, 2, 3])
    tfidf_comp = np.array([4, 5, 6])
    # Calculo el producto punto entre los dos vectores
    producto_punto = np.dot(tfidf_ref, tfidf_comp)
    # Calculo las normas de los vectores
    norma_vector1 = norm(tfidf_ref)
    norma_vector2 = norm(tfidf_comp)
    # Calculo la similitud coseno
    similitud_coseno = producto_punto / (norma_vector1 * norma_vector2)

    return similitud_coseno


def ordenar_por_similitud_cos(corpus, idx_doc):
    ref = corpus[idx_doc]
    print('Comparando con documento:', ref)
    del corpus[idx_doc]
    cosenos = {}
    for doc in corpus:
        cosenos[doc] = calcSimCos(tfidfs[ref],tfidfs[doc])
    
    # Ordeno el diccionario
    diccionario_ordenado = sorted(cosenos.items(), key=lambda x: x[1], reverse=True)
    return list(cosenos.keys())

In [517]:
idx_doc = 1
documentos_ordenados = ordenar_por_similitud_cos(list(corpus), idx_doc)

print(documentos_ordenados)


Comparando con documento: martes el dia de hoy es martes
['que dia es hoy', 'martes muchas gracias']
